In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [ ]:
# Path to your zip file (adjust if different)
zip_path = '/content/drive/MyDrive/SDAI/intel/archive.zip'
extract_path = '/content/classify_data'

In [ ]:
import zipfile
import os

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/data")

In [ ]:
import shutil

# Create YOLOv8 classification folder structure
os.makedirs("/content/classify_data/train", exist_ok=True)
os.makedirs("/content/classify_data/val", exist_ok=True)

shutil.move("/content/data/seg_train/seg_train", "/content/classify_data/train")
shutil.move("/content/data/seg_test/seg_test", "/content/classify_data/val")


'/content/classify_data/val/seg_test'

In [ ]:
import os
import shutil

# Define paths
base_path = "/content/classify_data"
train_src = os.path.join(base_path, "train", "seg_train")
val_src = os.path.join(base_path, "val", "seg_test")
train_dst = os.path.join(base_path, "train")
val_dst = os.path.join(base_path, "val")

# Move all class folders up one level from seg_train and seg_test
for cls_name in os.listdir(train_src):
    shutil.move(os.path.join(train_src, cls_name), os.path.join(train_dst, cls_name))
shutil.rmtree(train_src)

for cls_name in os.listdir(val_src):
    shutil.move(os.path.join(val_src, cls_name), os.path.join(val_dst, cls_name))
shutil.rmtree(val_src)


In [ ]:
import os

def print_dir_structure(root, max_level=2, indent=""):
    if max_level < 0:
        return
    try:
        entries = os.listdir(root)
    except NotADirectoryError:
        print(indent + os.path.basename(root) + " [FILE]")
        return
    print(indent + os.path.basename(root) + "/")
    for entry in entries:
        print_dir_structure(os.path.join(root, entry), max_level-1, indent + "    ")

print_dir_structure("/content/classify_data", max_level=2)


classify_data/
    val/
        sea/
        buildings/
        mountain/
        glacier/
        street/
        forest/
    train/
        sea/
        buildings/
        mountain/
        glacier/
        street/
        forest/


In [ ]:

from ultralytics import YOLO

!yolo classify train model=yolov8n-cls.pt data=/content/classify_data epochs=10 imgsz=224 batch=16




Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 5.31M/5.31M [00:00<00:00, 253MB/s]
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/classify_data, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_jso

In [ ]:
from pathlib import Path

# Load the trained model
model = YOLO('/content/runs/classify/train/weights/best.pt')

# Folder with
pred_path = Path("/content/data/seg_pred/seg_pred")
image_files = list(pred_path.glob("*.jpg"))

# Inference
for img_path in image_files:
    result = model.predict(source=img_path, imgsz=224)
    top1 = result[0].probs.top1
    label = result[0].names[top1]
    confidence = result[0].probs.data[top1].item()
    print(f"{img_path.name}: {label} ({confidence:.2f})")


Streaming output truncated to the last 5000 lines.

image 1/1 /content/data/seg_pred/seg_pred/23483.jpg: 224x224 glacier 1.00, mountain 0.00, sea 0.00, buildings 0.00, forest 0.00, 3.0ms
Speed: 2.2ms preprocess, 3.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
23483.jpg: glacier (1.00)

image 1/1 /content/data/seg_pred/seg_pred/2861.jpg: 224x224 sea 1.00, glacier 0.00, forest 0.00, mountain 0.00, street 0.00, 2.9ms
Speed: 2.2ms preprocess, 2.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
2861.jpg: sea (1.00)

image 1/1 /content/data/seg_pred/seg_pred/9178.jpg: 224x224 forest 1.00, glacier 0.00, mountain 0.00, sea 0.00, street 0.00, 3.0ms
Speed: 2.2ms preprocess, 3.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
9178.jpg: forest (1.00)

image 1/1 /content/data/seg_pred/seg_pred/3052.jpg: 224x224 glacier 1.00, mountain 0.00, street 0.00, sea 0.00, forest 0.00, 3.3ms
Speed: 2.2ms preprocess, 3.3ms inference, 0.0ms postproces

In [ ]:
!cp /content/runs/classify/train/weights/best.pt /content/drive/MyDrive/yolov8_class_model.pt
 # Download Model weights